In [12]:
import random
import requests
import os
from bs4 import BeautifulSoup
from bs4.element import NavigableString

import time
from tqdm import tqdm
import pandas as pd

In [4]:
def mine_poezja_org(url: str, author: str):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        links = [a.get('href') for a in soup.find_all('a') if a.get('href').startswith(url) and not a.get('href').startswith(f"{url}#")]
        if not os.path.exists(author):
            os.makedirs(author)

        content = []
        for link in tqdm(links):
            try:
                page = requests.get(link).content
                poem_soup = BeautifulSoup(page, 'html.parser')
                title = poem_soup.title.text
                poem = poem_soup.find(attrs={"itemprop": "text"}).text
                if "Å" in poem:
                    poem = poem.encode('cp1252').decode()

                if "english" not in title and "esperanto" not in title and "[en]" not in title:
                    poem = '\n'.join([line.strip() for line in poem.split('\n') if len(line.split(' ')) > 1])
                    lines_length = [len(line) for line in poem.split('\n')]
                    poem += '\n'
                    poem.replace("e", "e")
                    if max(lines_length) < 80:
                        content.append(poem)

            except requests.exceptions.ConnectionError:
                print("Failed to connect with %s." % link)
        dataframe = pd.DataFrame(content)
        dataframe.to_csv(f"data/poems/pl/{author}.csv", encoding='UTF-8', index=False, header=False)


# mine_poezja_org('https://poezja.org/wz/Miron_Bialoszewski/', 'bialoszewski')
# mine_poezja_org('https://poezja.org/wz/Krzysztof_Kamil_Baczynski/', 'baczynski')
# mine_poezja_org('https://poezja.org/wz/Zbigniew_Herbert/', 'herbert')
# mine_poezja_org('https://poezja.org/wz/Bolesław_Lesmian/', 'lesmian')
# mine_poezja_org('https://poezja.org/wz/Wislawa_Szymborska/', 'szymborska')
# mine_poezja_org('https://poezja.org/wz/Adam_Mickiewicz/', 'mickiewicz')
mine_poezja_org('https://poezja.org/wz/Jan_Kochanowski/', 'kochanowski')
mine_poezja_org('https://poezja.org/wz/Juliusz_Slowacki/', 'slowacki')

100%|██████████| 538/538 [01:41<00:00,  5.29it/s]

Saved 536 poems


In [16]:
def mine_azlyrics(url, author, skip:int = 0, _max: int = None):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        links = [a.get('href') for a in soup.find_all('a')]
        links = [href for href in links if href and href.startswith("/lyrics/")]
        content = []

        for link in tqdm(links[skip:skip + _max] if _max else links[skip:]):
            time.sleep(random.randint(1, 10))
            try:
                page = requests.get(f"https://www.azlyrics.com{link}").content
                poem_soup = BeautifulSoup(page, 'html.parser')
                divs = poem_soup.find_all('div')
                index = divs.index(poem_soup.find(attrs={"class": "ringtone"})) + 1
                text_lines = [str(content) for content in divs[index].contents if isinstance(content, NavigableString)][2:]
                text_lines = [line.strip() for line in text_lines if line and line != '\n']
                song = '\n'.join(text_lines) + '\n'
                content.append(song)
            except requests.exceptions.ConnectionError:
                print("Failed to connect with %s." % link)
            dataframe = pd.DataFrame(content)
            dataframe.to_csv(f"data/poems/pl/{author}.csv", encoding='UTF-8', index=False, header=False)

mine_azlyrics("https://www.azlyrics.com/t/tacohemingway.html", "tacohemingway", skip=8)
mine_azlyrics("https://www.azlyrics.com/o/ostr.html", "ostr", _max=30)

100%|██████████| 30/30 [03:19<00:00,  6.66s/it]
